In [3]:
!pip install psycopg2-binary pandas sqlalchemy yfinance PyPortfolioOpt

import psycopg2
import sqlalchemy
import pandas as pd
import csv
from sqlalchemy import create_engine
import yfinance as yf
import pandas as pd

In [4]:
# CONEXIÓN
DB_USER = "bancolombia"
DB_PASS = "pruebatecnica"
DB_HOST = "postgresql-bancolombia.alwaysdata.net"
DB_PORT = "5432"
DB_NAME = "bancolombia_prueba"

DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

DB_CONFIG = {
    "host": DB_HOST,
    "user": DB_USER,
    "password": DB_PASS,
    "dbname": DB_NAME
}

def ejecutar_consulta(sql_query):
    try:
        engine = create_engine(DATABASE_URL)
        df = pd.read_sql_query(sql_query, engine)
        return df
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return None
    finally:
        if 'engine' in locals():
            engine.dispose()

def ejecutar_modificacion(sql_command):
    try:
        with psycopg2.connect(**DB_CONFIG) as conexion:
            with conexion.cursor() as cursor:
                cursor.execute(sql_command)
        
        print(f"Comando ejecutado exitosamente.")
        return True
    except (Exception, psycopg2.Error) as error:
        print(f"Ocurrió un error al ejecutar el comando: {error}")
        return False





In [5]:
# Portafolio agregado
crear_funcion_agregada_sql = """
-- Crea una nueva función que devuelve el portafolio agregado Y su distribución porcentual
CREATE OR REPLACE FUNCTION obtener_portafolio_distribucion(cliente_id VARCHAR)
RETURNS TABLE (
    id_sistema_cliente VARCHAR,
    tipo_portafolio TEXT,
    identificador_activo TEXT,
    nombre_activo VARCHAR,
    valor_total_inversion NUMERIC,
    distribucion_portafolio NUMERIC -- Nueva columna para el porcentaje
) AS $$
BEGIN
    RETURN QUERY
        -- PASO 1: Usamos un CTE para agregar el portafolio como antes
        WITH portafolio_agregado AS (
            -- Aquí dentro va tu consulta de agregación que ya funciona
            SELECT
                pc.id_sistema_cliente,
                pc.tipo_portafolio,
                pc.identificador_activo,
                pc.nombre_activo,
                SUM(pc.valor_inversion) AS valor_total_inversion
            FROM
                ( -- Subconsulta para obtener el portafolio completo
                    SELECT
                        h_macro.id_sistema_cliente,
                        'Local COP' AS tipo_portafolio,
                        h_macro.cod_activo::TEXT AS identificador_activo,
                        cat_act.activo AS nombre_activo,
                        h_macro.aba AS valor_inversion
                    FROM historico_aba_macroactivos h_macro
                    LEFT JOIN catalogo_activos cat_act ON h_macro.cod_activo = cat_act.cod_activo
                    WHERE h_macro.id_sistema_cliente = cliente_id AND h_macro.ingestion_date = (SELECT MAX(sub.ingestion_date) FROM historico_aba_macroactivos sub)
                    UNION ALL
                    SELECT
                        h_usd.id_sistema_cliente,
                        'Internacional USD' AS tipo_portafolio,
                        h_usd.cusip AS identificador_activo, 
                        h_usd.nombre_activo,
                        h_usd.valor_mercado AS valor_inversion
                    FROM historico_aba_usd_internacional h_usd
                    WHERE h_usd.id_sistema_cliente = cliente_id AND h_usd.ingestion_date = (SELECT MAX(sub.ingestion_date) FROM historico_aba_usd_internacional sub)
                ) pc
            GROUP BY
                pc.id_sistema_cliente, pc.tipo_portafolio, pc.identificador_activo, pc.nombre_activo
        )
        -- PASO 2: Calculamos la distribución usando una Función de Ventana
        SELECT
            pa.id_sistema_cliente,
            pa.tipo_portafolio,
            pa.identificador_activo,
            pa.nombre_activo,
            pa.valor_total_inversion,
            -- LA MAGIA: (Valor del activo / Suma de todos los activos) * 100
            (pa.valor_total_inversion / SUM(pa.valor_total_inversion) OVER ()) * 100 AS distribucion_portafolio
        FROM
            portafolio_agregado pa
        ORDER BY
            distribucion_portafolio DESC;
END;
$$ LANGUAGE plpgsql;
"""
ejecutar_modificacion(crear_funcion_agregada_sql)

def obtener_portafolio_agregado_db(id_cliente):
    query = f"SELECT * FROM obtener_portafolio_distribucion('{id_cliente}');"
    
    df_portafolio = ejecutar_consulta(query)
    return df_portafolio

portafolio_agregado = obtener_portafolio_agregado_db('1004870235')

obtener_portafolio_agregado_db('10014876058')

Comando ejecutado exitosamente.


,id_sistema_cliente,tipo_portafolio,identificador_activo,nombre_activo,valor_total_inversion,distribucion_portafolio
0,10014876058,Local COP,1012,ISA,8704400.00,79.464823
1,10014876058,Internacional USD,91282CJK8,UNITED STATES TREAS NTS 4.625% 11/15/26 B/E DT...,348183.50,3.178661
2,10014876058,Internacional USD,91282CED9,UNITED STATES TREAS NTS 1.750% 03/15/25 B/E DT...,291774.00,2.663684
3,10014876058,Internacional USD,MONEYMRKT,BNY MELLON US DOLLAR LIQD SERV,150962.70,1.378179
4,10014876058,Internacional USD,91282CJE2,UNITED STATES TREAS NTS 5.000% 10/31/25 B/E DT...,149713.50,1.366775
5,10014876058,Internacional USD,L104AG738,BGF GLOBAL MULTI ASSET INCOME FUND CLASS A2 (U...,110692.85,1.010545
6,10014876058,Internacional USD,91282CGU9,UNITED STATES TREAS NTS 3.875% 03/31/25 B/E DT...,98878.00,0.902684
7,10014876058,Internacional USD,91282CDZ1,UNITED STATES TREAS NTS 1.500% 02/15/25 B/E DT...,97317.00,0.888433
8,10014876058,Internacional USD,USD999997,CASH,97000.00,0.885539
9,10014876058,Internacional USD,912828P46,UNITED STATES TREAS NTS 1.625% 02/15/26 B/E DT...,94438.00,0.862150


In [6]:
# Encontrar los retornos de los activos internacionales
query_identificadores_internacionales = """
SELECT DISTINCT
    simbol,
    cusip,
    isin
FROM
    historico_aba_usd_internacional
WHERE
    (simbol IS NOT NULL AND simbol != '')
    OR
    (cusip IS NOT NULL AND cusip != '')
    OR
    (isin IS NOT NULL AND isin != '');
"""

print("Obteniendo la lista de activos internacionales únicos...")
ejecutar_consulta(query_identificadores_internacionales)


Obteniendo la lista de activos internacionales únicos...


,simbol,cusip,isin
0,None,L4058R217,LU0109391861
1,BAK4918586,P1850NAA9,USP1850NAA92
2,PLTR,69608A108,US69608A1088
3,nan,P139234M9,XS2580672231
4,GOOG,02079K107,US02079K1079
...,...,...,...
118,None,L5785X484,LU1041599405
119,None,L8145U257,LU0181495838
120,IYK,464287812,US4642878122
121,None,L54486425,LU2114231884


In [7]:
df_identificadores = ejecutar_consulta(query_identificadores_internacionales)

def obtener_retornos_historicos(df_identificadores, periodo="2y"):
    if df_identificadores is None or df_identificadores.empty:
        print("El DataFrame de identificadores está vacío. No se pueden obtener retornos.")
        return None

    lista_tickers = []
    for index, row in df_identificadores.iterrows():
        if pd.notna(row['simbol']) and row['simbol'] != '':
            lista_tickers.append(row['simbol'])
        
    
    lista_tickers_unicos = list(set(lista_tickers)) 

    if not lista_tickers_unicos:
        print("No se encontraron tickers válidos en la lista de identificadores.")
        return None

    print(f"Intentando descargar datos de precios para {len(lista_tickers_unicos)} tickers...")

    precios_historicos = yf.download(lista_tickers_unicos, period=periodo, auto_adjust=True)['Close']
    
    precios_historicos.dropna(axis='columns', how='all', inplace=True)
    
    if precios_historicos.empty:
        print("No se pudieron obtener datos de precios para ninguno de los tickers.")
        return None
        
    print(f"Se obtuvieron datos exitosamente para {precios_historicos.shape[1]} tickers.")

    retornos_diarios = precios_historicos.pct_change().dropna()
    
    return retornos_diarios


df_retornos = obtener_retornos_historicos(df_identificadores)
print(df_retornos)


Intentando descargar datos de precios para 54 tickers...


[****                   8%                       ]  4 of 48 completedHTTP Error 404: 
[*******               15%                       ]  7 of 48 completedHTTP Error 404: 
[*********************100%***********************]  48 of 48 completed

16 Failed downloads:
['']: ValueError('Empty ticker name')
['EPMP5694987', 'BRK B', 'P9379RBC0', 'BGNB5166290', 'AVAL4362366', 'SQ', 'TEF5013179', 'BAK4918586', 'BGNB4479830', 'BNTM5022912', 'ECOH5689502', 'EPMP5013310', 'NONE', 'DLPG', 'ECOH5525375']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')


Se obtuvieron datos exitosamente para 32 tickers.
Ticker          AAPL      ABNB       AMD      AMZN      ARKK         C  \
Date                                                                     
2023-09-13 -0.011855 -0.018881  0.022790  0.025632 -0.013179  0.016555   
2023-09-14  0.008782 -0.004112 -0.010027 -0.000898  0.004145  0.018173   
2023-09-15 -0.004154 -0.009506 -0.048204 -0.029920 -0.012612 -0.009968   
2023-09-18  0.016913 -0.001401  0.008671 -0.002920 -0.024385 -0.001639   
2023-09-19  0.006181 -0.004911 -0.007424 -0.016788 -0.011902  0.000938   
...              ...       ...       ...       ...       ...       ...   
2025-09-05 -0.000375 -0.012443 -0.065826 -0.014214  0.004666 -0.017305   
2025-09-08 -0.007551  0.006542  0.001786  0.015108  0.010084  0.004927   
2025-09-09 -0.014839 -0.006500  0.029126  0.010176  0.006042  0.015333   
2025-09-10 -0.032259 -0.003231  0.023874 -0.033202 -0.013448 -0.001746   
2025-09-11  0.014286 -0.002917 -0.024257 -0.001650  0.036263  

/tmp/ipykernel_67138/3998122445.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos_diarios = precios_historicos.pct_change().dropna()


In [8]:
# TRABAJAR CON LOS ARCHIVOS PROBLEMATICOS

mapeo_locales = {
    'ISA': 'ISA.CL', 'PFCEMARGOS': 'PFCEMARGOS.CL', 'PFCORFICOL': 'PFCORFICOL.CL',
    'PFGRUPSURA': 'PFGRUPSURA.CL', 'GRUBOLIVAR': 'GRUBOLIVAR.CL', 'CELSIA': 'CELSIA.CL',
    'CEMARGOS': 'CEMARGOS.CL', 'ECOPETROL': 'ECOPETROL.CL', 'ETB': 'ETB.CL',
    'BRK B': 'BRK-B', 'SQ': 'SQ30.F'
}

mapeo_proxies = {
    'Fiducuenta': 'SHY', 'Plan Semilla': 'SHY', 'Renta Liquidez': 'SHY',
    'Fondo Cerrado Renta Fija IV': 'SHY', 'Fondo Cerrado Renta Fija V': 'SHY',
    'Fidurenta': 'ICOLCAP.CL', 'Sura Credito Privado': 'ICOLCAP.CL',
    'BAK4918586': '^GSPC', 'BGNB4479830': '^GSPC', 'EPMP5694987': '^GSPC',
    'NONE': '^GSPC', 'DLPG': '^GSPC', 'ECOH5689502': '^GSPC', 'TEF5013179': '^GSPC',
    'P9379RBC0': '^GSPC', 'ECOH5525375': '^GSPC', 'BNTM5022912': '^GSPC',
    'EPMP5013310': '^GSPC', 'AVAL4362366': '^GSPC', 'BGNB5166290': '^GSPC',
    'CDT TUYA TF B360': 'SHY',          
    'CDT BANCOLOMBIA B360': 'SHY',   
    'B': '^GSPC'
}

tickers_internacionales_reales = df_retornos.columns.tolist()
tickers_locales_reales = list(mapeo_locales.values())
tickers_de_proxies = list(mapeo_proxies.values())

lista_completa_tickers = tickers_internacionales_reales + tickers_locales_reales + tickers_de_proxies

lista_unica_tickers = sorted(list(set(lista_completa_tickers)))
df_lista_tickers = pd.DataFrame(lista_unica_tickers, columns=['simbol'])

print(f"Se ha creado una lista unificada con {len(lista_unica_tickers)} tickers únicos para descargar.")

df_retornos_diarios = obtener_retornos_historicos(df_lista_tickers)
print(df_retornos_diarios)

df_retornos_final = pd.DataFrame(index=df_retornos_diarios.index)

for ticker in df_retornos.columns:
    if ticker in df_retornos_diarios.columns:
        df_retornos_final[ticker] = df_retornos_diarios[ticker]

for nombre_original, ticker_real in mapeo_locales.items():
    if ticker_real in df_retornos_diarios.columns:
        df_retornos_final[nombre_original] = df_retornos_diarios[ticker_real]

for nombre_original, ticker_proxy in mapeo_proxies.items():
    if ticker_proxy in df_retornos_diarios.columns:
        df_retornos_final[nombre_original] = df_retornos_diarios[ticker_proxy]

print(f"DataFrame final 'traducido' construido con {df_retornos_final.shape[1]} activos.")

[                       0%                       ]

Se ha creado una lista unificada con 46 tickers únicos para descargar.
Intentando descargar datos de precios para 46 tickers...


[*********************100%***********************]  46 of 46 completed

Se obtuvieron datos exitosamente para 46 tickers.
Ticker          AAPL      ABNB       AMD      AMZN      ARKK     BRK-B  \
Date                                                                     
2024-09-30  0.022872 -0.011228 -0.001643 -0.008725 -0.007310  0.006099   
2024-10-01 -0.029141 -0.010567 -0.026390 -0.006440 -0.028403 -0.006192   
2024-10-02  0.002520  0.005659  0.000188 -0.001999 -0.006713 -0.001618   
2024-10-03 -0.004895 -0.007053  0.019214 -0.015155 -0.010464 -0.008124   
2024-10-04  0.005007  0.036396  0.049432  0.025005  0.025556  0.019891   
...              ...       ...       ...       ...       ...       ...   
2025-09-05 -0.000375 -0.012443 -0.065826 -0.014214  0.004666 -0.014085   
2025-09-08 -0.007551  0.006542  0.001786  0.015108  0.010084 -0.011985   
2025-09-09 -0.014839 -0.006500  0.029126  0.010176  0.006042 -0.002147   
2025-09-10 -0.032259 -0.003231  0.023874 -0.033202 -0.013448 -0.005358   
2025-09-11  0.014286 -0.002917 -0.024257 -0.001650  0.036263  


/tmp/ipykernel_67138/3998122445.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos_diarios = precios_historicos.pct_change().dropna()


In [9]:
def calcular_inputs_modelo(df_retornos_diarios):
    if df_retornos_diarios is None or df_retornos_diarios.empty:
        print("El DataFrame de retornos está vacío. No se pueden calcular los inputs.")
        return None, None

    retornos_promedio_diarios = df_retornos_diarios.mean()

    #    Se multiplica por 252, el número aproximado de días de mercado en un año.
    retornos_esperados = retornos_promedio_diarios * 252

    # Calcular la matriz de covarianza de los retornos diarios
    matriz_covarianza_diaria = df_retornos_diarios.cov()

    # Anualizar la matriz de covarianza
    matriz_covarianza = matriz_covarianza_diaria * 252

    print("Inputs del modelo calculados exitosamente.")
    return retornos_esperados, matriz_covarianza

df_retornos_yfinance, matriz_cov = calcular_inputs_modelo(df_retornos_final)
print(df_retornos_yfinance)
print(matriz_cov)

Inputs del modelo calculados exitosamente.
AAPL                    0.065585
ABNB                    0.033426
AMD                     0.074091
AMZN                    0.260682
ARKK                    0.597166
                          ...   
AVAL4362366             0.157710
BGNB5166290             0.157710
CDT TUYA TF B360        0.035867
CDT BANCOLOMBIA B360    0.035867
B                       0.157710
Length: 66, dtype: float64
                          AAPL      ABNB       AMD      AMZN      ARKK  \
AAPL                  0.103522  0.065837  0.086700  0.065217  0.083557   
ABNB                  0.065837  0.153651  0.098401  0.071761  0.105877   
AMD                   0.086700  0.098401  0.265256  0.092019  0.147615   
AMZN                  0.065217  0.071761  0.092019  0.112398  0.101455   
ARKK                  0.083557  0.105877  0.147615  0.101455  0.196593   
...                        ...       ...       ...       ...       ...   
AVAL4362366           0.045709  0.047078  0.06901

In [10]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt.objective_functions import L2_reg

def optimizar_portafolio(portafolio_actual_df, retornos_esperados, matriz_covarianza):
    print("\n--- Iniciando Optimización de Portafolio ---")


    portafolio_actual_df = portafolio_actual_df.dropna(subset=['nombre_activo'])
    portafolio_actual_df = portafolio_actual_df[portafolio_actual_df['nombre_activo'] != '']

    activos_del_cliente = portafolio_actual_df['nombre_activo'].tolist()
    
    activos_conocidos = [activo for activo in activos_del_cliente if activo in retornos_esperados.index]
    activos_desconocidos = list(set(activos_del_cliente) - set(activos_conocidos))

    if not activos_conocidos:
        print("ERROR: Ninguno de los activos en el portafolio del cliente fue encontrado en los datos de mercado.")
        return None, 0, 0, 0 # Devuelve valores vacíos para evitar que el script se detenga
        
    if activos_desconocidos:
        print(f"ADVERTENCIA: Se ignorarán los siguientes activos por no tener datos de mercado: {activos_desconocidos}")

    
    mu = retornos_esperados.loc[activos_conocidos]
    S = matriz_covarianza.loc[activos_conocidos, activos_conocidos]
    
    print(f"Optimizando para {len(activos_conocidos)} activos conocidos en el portafolio...")

    ef = EfficientFrontier(mu, S)
    ef.add_objective(L2_reg) 
    
    ef.max_sharpe()
    pesos_limpios = ef.clean_weights()
    
    rendimiento, volatilidad, sharpe = ef.portfolio_performance(verbose=False)
    
    print("Optimización completada.")
    return pesos_limpios, rendimiento, volatilidad, sharpe




In [11]:
!pip install plotly

import plotly.graph_objects as go
import pandas as pd
import numpy as np


portafolio_actual = pd.DataFrame({
    "nombre_activo": ["GOOG", "PFGRUPSURA", "Fidurenta", "AMZN"],
    "valor_inversion": [40000, 30000, 20000, 10000]
})
total_invertido = portafolio_actual['valor_inversion'].sum()
portafolio_actual['distribucion_%'] = (portafolio_actual['valor_inversion'] / total_invertido) * 100


pesos_optimos_dict, rendimiento_opt, volatilidad_opt, sharpe_opt = optimizar_portafolio(
    portafolio_actual,
    df_retornos_yfinance,  
    matriz_cov    
)

df_recomendacion = pd.DataFrame(list(pesos_optimos_dict.items()), columns=['nombre_activo', 'distribucion_optima_%'])
df_recomendacion['distribucion_optima_%'] *= 100 # Convertir de 0.5 a 50%

# Se crea el DataFrame comparativo que usaremos para todo
df_comparativo = pd.merge(
    portafolio_actual[['nombre_activo', 'distribucion_%']],
    df_recomendacion,
    on='nombre_activo',
    how='outer'
).fillna(0)

recomendaciones_texto = []
for index, row in df_comparativo.iterrows():
    diferencia = row['distribucion_optima_%'] - row['distribucion_%']
    if diferencia > 0.1: # Un umbral para no mostrar cambios insignificantes
        recomendaciones_texto.append(f"<li>Aumentar la exposición en <b>{row['nombre_activo']}</b> en un <b>{diferencia:.2f}%</b> (de {row['distribucion_%']:.2f}% a {row['distribucion_optima_%']:.2f}%).</li>")
    elif diferencia < -0.1:
        recomendaciones_texto.append(f"<li>Reducir la exposición en <b>{row['nombre_activo']}</b> en un <b>{-diferencia:.2f}%</b> (de {row['distribucion_%']:.2f}% a {row['distribucion_optima_%']:.2f}%).</li>")

html_recomendaciones = "<ul>" + "".join(recomendaciones_texto) + "</ul>"


# Paleta de colores cálida (de amarillo a rojo)
colores_calidos = ['#fee8c8','#fdd49e','#fdbb84','#fc8d59','#ef6548','#d7301f','#b30000']

# Gráfico 1: Composición Actual
fig_torta = go.Figure(data=[go.Pie(
    labels=portafolio_actual['nombre_activo'],
    values=portafolio_actual['valor_inversion'],
    hole=.4,
    marker_colors=colores_calidos
)])
fig_torta.update_layout(title_text='Composición Actual del Portafolio')

# Gráfico 2: Comparativa
fig_barras = go.Figure(data=[
    go.Bar(name='Actual', x=df_comparativo['nombre_activo'], y=df_comparativo['distribucion_%'], marker_color='#fc8d59'),
    go.Bar(name='Recomendado', x=df_comparativo['nombre_activo'], y=df_comparativo['distribucion_optima_%'], marker_color='#d7301f')
])
fig_barras.update_layout(
    barmode='group',
    title_text='Distribución Actual vs. Recomendación Óptima (%)'
)


# Convertimos los gráficos a HTML sin incluir la librería completa en cada uno
html_torta = fig_torta.to_html(full_html=False, include_plotlyjs='cdn')
html_barras = fig_barras.to_html(full_html=False) 

# Creamos la plantilla HTML final
html_completo = f"""
<html>
<head>
    <title>Reporte de Optimización de Portafolio</title>
    <style>
        body {{ font-family: sans-serif; margin: 40px; }}
        .container {{ width: 100%; }}
        .chart {{ width: 100%; margin-bottom: 30px; }} 
        h1, h2 {{ color: #b30000; }}
        li {{ margin-bottom: 10px; }}
    </style>
</head>
<body>
    <h1>Optimización de Portafolio - Demo</h1>
    <p>Este reporte muestra el análisis del portafolio ficticio y la recomendación generada por el modelo de optimización de Markowitz para maximizar el Sharpe Ratio.</p>
    
    <hr>
    
    <h2>Recomendaciones Clave para Rebalanceo:</h2>
    {html_recomendaciones}
    
    <div class="container">
        <div class="chart">{html_torta}</div>
        <div class="chart">{html_barras}</div>
    </div>

    <h2>Resultados del Portafolio Optimizado:</h2>
    <ul>
        <li><b>Rendimiento Anual Esperado:</b> {rendimiento_opt:.2%}</li>
        <li><b>Volatilidad Anual (Riesgo):</b> {volatilidad_opt:.2%}</li>
        <li><b>Sharpe Ratio:</b> {sharpe_opt:.2f}</li>
    </ul>

</body>
</html>
"""

# Guardamos el resultado en un solo archivo
with open("demo_optimizacion_portafolio.html", "w") as f:
    f.write(html_completo)

print("\n✅ Reporte 'demo_optimizacion_portafolio.html' ha sido creado")


--- Iniciando Optimización de Portafolio ---
Optimizando para 4 activos conocidos en el portafolio...
Optimización completada.


/usr/local/python/3.12.1/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(



✅ Reporte 'demo_optimizacion_portafolio.html' ha sido creado
